# Introduction to Data Science 
# Lecture 9: Linear Regression 2
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll discuss:
* overfitting, model generalizability, and the bias-variance tradeoff
* cross validation 
* using categorical variables for regression  

Recommended reading:
* G. James, D. Witten, T. Hastie, and R. Tibshirani, An Introduction to Statistical Learning, Ch. 3 [digitial version available here](https://d1wqtxts1xzle7.cloudfront.net/60707896/An_Introduction_to_Statistical_Learning_with_Applications_in_R-Springer_201320190925-63943-2cqzhk-libre.pdf?1569480857=&response-content-disposition=inline%3B+filename%3DAn_Introduction_to_Statistical_Learning.pdf&Expires=1675807025&Signature=EpiMSZAPOQ6yVW2yAZzLWCYz0MUeqp04yzupocLwB5CLHj5ugPf90tXrC6unkVv96az1rCxOeNnNb0bJl5KTCRUJHt7vXtu8pSC7H5vUO7NFOKf7QAmq4Z8vvvpkhN29uWD~fidv3lTH~Glg4-qdaGKf~qNg0J1cqnJ-ibwZCWjTuReWC4DMsi~00dA76CL4woM1a3uMUYsiCQF9~qchsI~t9hUFmvBByql3eFhGgxU2IJBCMJ~sUk~fnhmK-aGITL1ATOTVIa4rT1ucm7lh64ROdsu5RhEv5VpxotU~JjYbwdAvqiNB~QjAiIRELzqYvMOKiccn8uL0POR5DAdPqQ__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA)


## Review from Lecture 8(Linear Regression 1)

### Simple Linear Regression (SLR)

**Data**: We have $n$ samples $(x, y)_i$, $i=1,\ldots n$. 

**Model**: $y \sim \beta_0 + \beta_1 x$ 

**Goal**: Find the best values of $\beta_0$ and $\beta_1$, denoted $\hat{\beta}_0$ and $\hat{\beta}_1$, so that the prediction $\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x$ "best fits" the data. 

<img src="438px-Linear_regression.png" width="40%" alt="https://en.wikipedia.org/wiki/Linear_regression">

**Theorem.** 
The parameters that minimize the "residual sum of squares (RSS)", 
$RSS = \sum_i (y_i - \beta_0 - \beta_1 x_i)^2$, 
are: 
$$
\hat{\beta}_1 = \frac{\sum_{i=1}^n (x_i - \overline{x})(y_i - \overline{y}) }{\sum_{i=1}^n (x_i - \overline{x})^2}
\qquad \textrm{and} \qquad
\hat{\beta}_0 = \overline{y} -  \hat{\beta}_1 \overline{x}. 
$$
where $\overline{x} = \frac{1}{n} \sum_{i=1}^n x_i$ and $\overline{y} = \frac{1}{n} \sum_{i=1}^n y_i$. 


### Multilinear regression 

**Data**: We have $n$ samples of the form $\big(x_1, x_2 , \ldots, x_m , y \big)_i$, $i=1,\ldots n$. 

**Model**: $y \sim \beta_0 + \beta_1 x_1 + \cdots + \beta_m x_m $ 

### Nonlinear relationships  

**Data**: We have $n$ samples $\big(x_1, x_2 , \ldots, x_m , y \big)_i$, $i=1,\ldots n$. 

**Model**: $y \sim \beta_0 + \beta_1 f_1(x_1,x_2,\ldots,x_m) + \cdots +  \beta_k f_k(x_1,x_2,\ldots,x_m)$ 


## Regression with python

There are several different python packages that do regression:
+ [statsmodels](http://statsmodels.sourceforge.net/)
+ [scikit-learn](http://scikit-learn.org/)
+ [SciPy](http://www.scipy.org/)
+ ... 

Note statsmodels approaches regression from a statistics viewpoint, while scikit-learn approaches it from a machine learning viewpoint. I'll say more about this today. 

SciPy has some regression tools, but compared to these other two packages, they are relatively limited. 


In [ ]:
# imports and setup

import scipy as sc
import numpy as np

import pandas as pd
import statsmodels.formula.api as sm     #Last lecture: used statsmodels.formula.api.ols() for OLS
from sklearn import linear_model         #Last lecture: used sklearn.linear_model.LinearRegression() for OLS

import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (7.5, 4.5)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

## Advertisement dataset
Consider the 'Advertising' dataset from
[here](http://www-bcf.usc.edu/~gareth/ISL/data.html).


For 200 different ‘markets’ (think different cities), this dataset consists of the number of sales of a particular product as well as the advertising budget for three different media: TV, radio, and newspaper. 

Last time, after trying a variety of linear models, we discovered the following one, which includes a nonlinear relationship between the TV budget and Radio budget:
$$
\text{Sales} = \beta_0 + \beta_1 * \text{TV_budget} + \beta_2*\text{Radio_budget} + \beta_3 * \text{TV_budget} *\text{Radio_budget}. 
$$

In [ ]:
advert = pd.read_csv('Advertising.csv',index_col=0) #load data

ad_NL = sm.ols(formula="Sales ~ TV + Radio + TV*Radio", data=advert).fit()
ad_NL.summary()

This model is really excellent: 
- $R^2 = 97\%$ of the variability in the data is accounted for by the model. 
- The $p$-value for the F-statistic is very small. 
- The $p$-values for the individual coefficients are small. 

Interpretation: 
- In a particular market, if I spend an additional $1k on TV advertising, what do I expect sales to do? 
- Should I spend additional money on TV or Radio advertising? 

In [ ]:
#Initialize figure and axes for 3d plot:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d') 

#Make 3d scatter plot of the data:
ax.scatter(xs=advert['TV'], ys=advert['Radio'], zs=advert['Sales']) 

#Make a surface plot of the PREDICTED response variable
x = np.linspace(advert['TV'].min(), advert['TV'].max(), 100)
y = np.linspace(advert['Radio'].min(), advert['Radio'].max(), 100)
X,Y = np.meshgrid(x,y) #Create grid of values for explanatory variables
par = dict(ad_NL.params) #Make a dictionary out of model coefficients
print(par)
Z = par["Intercept"] + par["TV"]*X + par["Radio"]*Y + par["TV:Radio"]*X*Y
surf = ax.plot_surface(X, Y, Z,cmap=cm.Greys, alpha=0.2) #alpha<1 makes surface plot transparent

# Rotate and label axes:
ax.view_init(25,-71)
ax.set_xlabel('TV budget')
ax.set_ylabel('Radio budget')
ax.set_zlabel('Sales')

plt.show()

### A word of caution on overfitting

It is tempting to include a lot of terms in the regression, but this is problematic. A useful model will  *generalize* beyond the data given to it. 


**Questions?**

## Overfitting, underfitting, model generalizability, and the bias–variance tradeoff

In regression, and other prediction problems, we would like to develop a model on a dataset, that would preform well, not only on that dataset, but on similar data that the model hasn't yet seen. If a model satisfies this criterion, we say that it is *generalizable*. 

Consider the following data, that has been fit with a linear polynomial model (black) and a high degree polynomial model (blue). For convenience, let me call these the black and blue models, respectively. 

<img src="overfitted_data.png" title="https://commons.wikimedia.org/w/index.php?curid=47471056" width="40%">

Let's call the dataset that we train the model on the *training dataset* and the dataset that we test the model on the *testing dataset*. In the above figure, the training dataset are the black points and the testing dataset is not shown, but we imagine it to be similar to the points shown. 

Which model is better? 

The blue model has 100% accuracy on the training dataset, while the black model has much smaller accuracy. However, the blue model is highly oscillatory and might not generalize well to new data. For example, the model would wildly miss the test point $(3,0)$. We say that the blue model has *overfit* the data. On the other hand, it isn't difficult to see that we could also *underfit* the data. In this case, the model isn't complex enough to have good accuracy on the training dataset. 

This phenomena is often described in terms of the *bias-variance tradeoff*. Here, we decompose the error of the model into three terms:
$$
\textrm{Error} = 
\textrm{Bias} + 
\textrm{Variance} + 
\textrm{Irreducible Error}. 
$$
- The *bias* of the method is the error caused by the simplifying assumptions built into the method. 
+ The *variance* of the method is how much the model will change based on the sampled data. 
+ The *irreducible error* is error in the data itself, so no model can capture this error. 

There is a tradeoff between the bias and variance of a model. 
High-variance methods (e.g., the blue method) are accurate on the training set, but overfit noise in the data, so don't generalized well to new data. High-bias models (e.g., the black method) are too simple to fit the data, but are better at generalizing to new test data. 


## Generalizability in practice

Consider the Auto dataset, which contains 9 features (mpg, cylinders, displacement, horsepower, weight, acceleration, year, origin, name) for 397 different used cars. This dataset is available in the lectures folder.

In [ ]:
auto = pd.read_csv('Auto.csv') #load data
print(auto.dtypes) #Horsepower should be an integer but imported as object

# some of the horsepowers are '?', so we just remove them and then map the remaining strings to integers
auto = auto[auto.horsepower != '?']
auto['horsepower'] = auto['horsepower'].map(int)

auto

In [ ]:
print(auto.describe())

Let's consider the relationship between mpg and horsepower.

In [ ]:
plt.scatter(auto['horsepower'],auto['mpg'],color='black',linewidth=1)
plt.xlabel('horsepower'); plt.ylabel('mpg')
plt.ylim((0,50))
plt.show()

We consider the linear model
$$
\text{mpg} = \beta_0 + \beta_1 \text{horsepower} + \beta_2 \text{horsepower}^2 + \cdots + \beta_m \text{horsepower}^m
$$
It might seem that choosing $m$ to be large would be a good thing. After all, a high degree polynomial is more flexible than a small degree polynomial. 

In [ ]:
# fit polynomial models
mr1 = sm.ols(formula="mpg ~ horsepower", data=auto).fit()
par1 = dict(mr1.params)
mr2 = sm.ols(formula="mpg ~ horsepower + I(horsepower ** 2.0)", data=auto).fit()
par2 = dict(mr2.params)
mr3 = sm.ols(formula="mpg ~ horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)", data=auto).fit()
par3 = dict(mr3.params)
mr4 = sm.ols(formula="mpg ~ horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0) + I(horsepower ** 4.0)", data=auto).fit()
par4 = dict(mr4.params)

# make scatterplot of data
plt.scatter(auto['horsepower'],auto['mpg'],color='black',label="data")

# evaluate polynomial models on a grid
x = np.linspace(0,250,1000)
y1 = par1["Intercept"] + par1['horsepower']*x
y2 = par2["Intercept"] + par2['horsepower']*x + par2['I(horsepower ** 2.0)']*x**2
y3 = par3["Intercept"] + par3['horsepower']*x + par3['I(horsepower ** 2.0)']*x**2 + par3['I(horsepower ** 3.0)']*x**3
y4 = par4["Intercept"] + par4['horsepower']*x + par4['I(horsepower ** 2.0)']*x**2 + par4['I(horsepower ** 3.0)']*x**3 + par4['I(horsepower ** 4.0)']*x**4

# plot polynomial models
plt.plot(x,y1,label="degree 1",linewidth=2)
plt.plot(x,y2,label="degree 2",linewidth=2)
plt.plot(x,y3,label="degree 3",linewidth=2)
plt.plot(x,y4,label="degree 4",linewidth=2)
plt.legend()
plt.xlabel('horsepower'); plt.ylabel('mpg')
plt.ylim((0,50))
plt.show()

In [ ]:
print('mr1:',mr1.rsquared)
print('mr2:',mr2.rsquared)
print('mr3:',mr3.rsquared)
print('mr4:',mr4.rsquared)

As $m$ increases, the $R^2$ value is becoming larger. (You can prove that this is always true if you add more predictors.)

Let's check the $p$-values for the coefficients for the degree 4 fit.

In [ ]:
mr1.summary()

In [ ]:
mr2.summary()

In [ ]:
mr3.summary()

In [ ]:
mr4.summary()

For $m>2$, the $p$-values are very large, so we don't have a strong relationship between the variables. 

We could rely on *Occam's razor* to decide between models. Occam's razor can be stated: among many different models that explain the data, the simplest one should be used. Since we don't get much benefit in terms of $R^2$ values by choosing $m>2$, we should use $m=2$. 

But there are even better criterion for deciding between models. 

## Cross-validation

There is a clever method for developing generalizable models that aren't underfit or overfit, called *cross validation*. 

**Cross-validation** is a general method for assessing how the results of a predictive model (regression, classification,...) will *generalize* to an independent data set. In regression, cross-validation is a method for assessing how well the regression model will predict the dependent value for points that weren't used to *train* the model. 

The idea of the method is simple: 
+ Split the dataset into two groups: the training dataset and the testing dataset. 
+ Train a variety of models on the training dataset. 
+ Check the accuracy of each model on the testing dataset. 
+ By comparing these accuracies, determine which model is best.

In practice, you have to decide how to split the data into groups (i.e. how large the groups should be). You might also want to repeat the experiment so that the assessment doesn't depend on the way in which you split the data into groups. We'll worry about these questions in a later lecture.

As the model becomes more complex ($m$ increases), the accuracy always increases for the training dataset. But, at some point, it starts to overfit the data and the accuracy decreases for the test dataset! Cross validation techniques will allow us to find the sweet-spot for the parameter $m$!

Let's see this concept for the relationship between mpg and horsepower in the Auto dataset. We'll use the scikit-learn package for the cross validation analysis instead of statsmodels, because it is much easier to do cross validation there. 

In [ ]:
lr = linear_model.LinearRegression() # create a linear regression object

# add additional columns to the dataframe for the various powers of horsepower
for m in np.arange(2,6): 
    auto['h'+str(m)] = auto['horsepower']**m

# with scikit-learn, we have to extract values from the pandas dataframe
X = auto[['horsepower','h2','h3','h4','h5']].values
y = auto['mpg'].values
print(X) #note X is a numpy array
print(type(X))

plt.scatter(X[:,0], y,  color='black',label='data')

# make data for plotting
xs = np.linspace(20, 250, num=100)
Xs = np.zeros([100,5])
Xs[:,0] = xs
for m in np.arange(1,5): 
    Xs[:,m] = xs**(m+1)
    
for m in np.arange(1,6):     
    lr.fit(X=X[:,:m], y=y)
    plt.plot(xs, lr.predict(X=Xs[:,:m]), linewidth=3, label = "m = " + str(m) )

plt.legend(loc='upper right')
plt.xlabel('horsepower'); plt.ylabel('mpg')
plt.ylim((0,50))
plt.show()

### Cross validation using scikit-learn 

- In scikit-learn, you can use the [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split the dataset into a training dataset and a test dataset.
+ The *score* function returns the coefficient of determination, $R^2$, of the prediction.

In the following code, I've split the data in an unusual way - taking the test set to be 90% - to illustrate the point more clearly. Typically, we might make the training set to be 70-80% of the dataset. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

plt.scatter(X_train[:,0], y_train,  color='red',label='training data')
plt.scatter(X_test[:,0], y_test,  color='black',label='test data')

for m in np.arange(1,6):     
    lr.fit(X=X_train[:,:m], y=y_train)
    print('m=', m, ', train: ', lr.score(X_train[:,:m], y_train), ' test: ', lr.score(X_test[:,:m], y_test))
    plt.plot(xs, lr.predict(X=Xs[:,:m]), linewidth=3, label = "m = " + str(m) )

plt.legend()
plt.xlabel('horsepower'); plt.ylabel('mpg')
plt.ylim((0,50))
plt.show()


We observe that as the model complexity increases, 
- the accuracy on the training data increases, but 
+ the generalizability of the model to the test set decreases. 

Our job as data analysts is to find a model that is sufficiently complex to describe the training data, but not so complex that it isn't generalizable to new data. 

## Class exercise: analysis of the credit dataset 

We'll use [Statsmodels](http://statsmodels.sourceforge.net/) to study a dataset related to credit cards.
We'll use the 'Credit' dataset stored in the lectures folder. 
This dataset consists of some credit card information for 400 people. 

Of course, a *credit card* is a card issued to a person ("cardholder"), typically from a bank, that can be used as a method of payment. The card allows the cardholder to borrow money from the bank to pay for goods and services. Credit cards have a *limit*, the maximum amount you can borrow, which is determined by the bank. The limit is determined from information collected from the cardholder (income, age, ...) and especially (as we will see) the cardholders "credit rating".  The *credit rating* is an evaluation of the (1) ability of the cardholder to pay back the borrowed money and (2) the likelihood of the cardholder to defaulting on the borrowed money. 

Our focus will be on the use of regression tools to study this dataset. Ideally, we'd like to understand what factors determine *credit ratings* and *credit limits*. We can think about this either from the point of view of (1) a bank who wants to protect their investments by minimizing credit defaults or (2) a person who is trying to increase their credit rating and/or credit limit. 

A difficulty we'll encounter is including categorical data in regression models.  

In [ ]:
# Import data from Credit.csv file
credit = pd.read_csv('Credit.csv',index_col=0) #load data
credit

In [ ]:
# Summarize and describe data

print(credit.dtypes, '\n') 
print(credit['Gender'].value_counts(), '\n')
print(credit['Student'].value_counts(), '\n')
print(credit['Married'].value_counts(), '\n')
print(credit['Ethnicity'].value_counts())
credit.describe()

The column names of this data are:  
+ Income 
+ Limit  
+ Rating  
+ Cards
+ Age  
+ Education  
+ Gender (categorial: M, F)
+ Student (categorial: Y, N)
+ Married (categorial: Y, N)
+ Ethnicity (categorial: Caucasian, Asian, African American) 
+ Balance

**Question:** What is wrong with the income data? How can it be fixed? 

The file 'Credit.csv' is a comma separated file. I assume a period was used instead of a comma to indicate thousands in income so it wouldn't get confused with the separating value? Or maybe this is a dataset from Europe? Or maybe the income is just measured in \$1k units?  To change the income data, we can use the Pandas series 'map' function. 


In [ ]:
credit["Income"] = credit["Income"].map(lambda x: 1000*x)
print(credit[:10])

We can also look at the covariances in the data. (This is how the variables vary together.) There are two ways to do this:
+ Quantitatively: Compute the correlation matrix. For each pair of variables, $(x_i,y_i)$, we compute 
$$
\frac{\sum_i (x_i - \bar x) (y_i - \bar y)}{s_x s_y}
$$ 
where $\bar x, \bar y$ are sample means and $s_x, s_y$ are sample variances. 
+ Visually: Make a scatter matrix of the data


In [ ]:
credit.corr()

In [ ]:
# trick: semi-colon prevents unwanted output
pd.plotting.scatter_matrix(credit, figsize=(10, 10), diagonal='hist');


**Observations:**
+ Limit and Rating are highly correlated ($99.7\%$)  
+ Income strongly correlates with Limit ($79\%$) and Rating ($79\%$)
+ Balance correlates with Limit ($86\%$) and Rating ($86\%$)
+ There are "weird stripes" in some of the data. Why? 
+ Categorical information doesn't appear in this plot. Why? How can I visualize the categorical variables?

In [ ]:
# Plot Categorical variables: Gender, Student, Married, Ethnicity

# Barplots
fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(8,8))
credit["Gender"].value_counts().plot(kind='bar',ax=axes[0,0],title='Gender');
credit["Student"].value_counts().plot(kind='bar',ax=axes[1,0],title='Student');
credit["Married"].value_counts().plot(kind='bar',ax=axes[0,1],title='Married');
credit["Ethnicity"].value_counts().plot(kind='bar',ax=axes[1,1],title='Ethnicity');
fig.suptitle('Barplots of Various Categorical Variables',fontsize=14)
plt.show()

# Side-by-side boxplots for Limit by categories
fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(8,8))
credit.boxplot(column=['Limit'], by=['Gender'],ax=axes[0,0])
credit.boxplot(column=['Limit'], by=['Student'],ax=axes[0,1])
credit.boxplot(column=['Limit'], by=['Married'],ax=axes[1,0])
credit.boxplot(column=['Limit'], by=['Ethnicity'],ax=axes[1,1])
fig.suptitle('Boxplots of Credit Limit by Various Categorical Variables', fontsize=14)
plt.show()

## Incorporating categorical variables into regression models

We have four categorical variables (Gender, Student, Married, Ethnicity). How can we include them in a regression model? 

Let's start with a categorical variable with only 2 categories: Gender (Male, Female).

Idea: Create a "dummy variable" that turns Gender into a real value: 
$$
\text{Gender_num}_i = \begin{cases} 
1 & \text{if $i$-th person is female} \\
0 & \text{if $i$-th person is male}
\end{cases}. 
$$
Then we could try to fit a model of the form
$$
\text{Income} = \beta_0 + \beta_1 \text{Gender_num}. 
$$

In [ ]:
credit["Gender_num"] = credit["Gender"].map({' Male':0, 'Female':1})
credit["Student_num"] = credit["Student"].map({'Yes':1, 'No':0})
credit["Married_num"] = credit["Married"].map({'Yes':1, 'No':0})

credit_model = sm.ols(formula="Income ~ Gender_num", data=credit).fit()
credit_model.summary()

Since the $p$-value for the Gender_num coefficient is very large, there is no support for the conclusion that there is a difference in income between genders.


## What about a categorical variable with 3 categories? 

The Ethnicity variable takes three values: Caucasian, Asian, and African American. 

What's wrong with the following?  
$$
\text{Ethnicity_num}_i = \begin{cases} 
0 & \text{if $i$-th person is Caucasian} \\
1 & \text{if $i$-th person is Asian} \\ 
2 & \text{if $i$-th person is African American}
\end{cases}. 
$$


We'll need more than one dummy variable:  
$$
\text{Asian}_i = \begin{cases} 
1 & \text{if $i$-th person is Asian} \\
0 & \text{otherwise}
\end{cases}. 
$$
$$
\text{Caucasian}_i = \begin{cases} 
1 & \text{if $i$-th person is Caucasian} \\
0 & \text{otherwise}
\end{cases}. 
$$
The value with no dummy variable--African American--is called the *baseline*.

We can use the *get_dummies* function to automatically get these values

In [ ]:
dummy = pd.get_dummies(credit['Ethnicity'])
credit = pd.concat([credit,dummy],axis=1)
credit

![image](https://imgs.xkcd.com/comics/error_bars_2x.png)
